In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import random

# Membaca dataset
df_selected = pd.read_csv('dataset-berlabel-aspek.csv')

# Memuat stopwords
def load_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return set(file.read().splitlines())
stopwords2 = load_file('txt/stopwords-2.txt')

# Mengahapus stopwords
def preprocessing(text, stopwords):
    text = [word for word in text.split() if word not in stopwords]
    return ' '.join(text) 

# Menghapus stopwords dari kolom 'teks' dan menyimpannya dalam kolom baru 'teks-kmeans'
df_selected['teks-kmeans'] = df_selected['teks'].apply(lambda x: preprocessing(x, stopwords2))

centroid_sentences = {
    'kompensasi': "gaji kompensasi",
    'kepuasan_kerja': "mental stres jam",
    'aktualisasi': "berkembang kembang jabatan skill",
    'hubungan_kerja': "hubungan jahat hubungan baik"
}
# Menghitung posisi dalam DataFrame untuk setiap centroid
num_rows = len(df_selected)
posisi = {
    int(num_rows * 0.25): centroid_sentences['kompensasi'],
    int(num_rows * 0.50): centroid_sentences['kepuasan_kerja'],
    int(num_rows * 0.75): centroid_sentences['aktualisasi'],
    int(num_rows * 0.90): centroid_sentences['hubungan_kerja']
}
# Menyisipkan kalimat ke dalam DataFrame pada posisi yang ditentukan
for pos, sentence in posisi.items():
    df_selected.at[pos, 'teks-kmeans'] = sentence
# Vektorisasi teks menggunakan TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_selected['teks-kmeans'])  # Menggunakan kolom teks yang telah dibersihkan
lokasi_centroid = X[list(posisi.keys())].toarray()

# K-means clustering
kmeans = KMeans(n_clusters=4, init=lokasi_centroid, n_init=10, random_state=0)
kmeans.fit(X)

# Menyimpan hasil klaster pada kolom baru 'skor-klaster-prediksi'
df_selected['skor-klaster-prediksi'] = kmeans.labels_

db_score = davies_bouldin_score(X.toarray(), kmeans.labels_)
print(f"Davies-Bouldin Score: {db_score:.2f}")

# Memisahkan klaster menjadi DataFrame yang berbeda dan menambahkan kolom 'label'
clusters = [df_selected[df_selected['skor-klaster-prediksi'] == i][['teks-kmeans', 'label']].reset_index(drop=True) for i in range(4)]

# Label untuk setiap klaster
label_klaster = ['kompensasi', 'kepuasan kerja', 'aktualisasi', 'hubungan kerja']

# Menampilkan dan menyimpan hasil
for label, cleaned_data in zip(label_klaster, clusters):  # Menyesuaikan penggunaan zip
    print(f"Faktor {label.capitalize()}:")
    print(cleaned_data[['teks-kmeans', 'label']])

    # Menyimpan data ke file
    cleaned_data.to_csv(f'klaster/{label}.txt', sep='\t', index=False, header=True)